# Scraper de Avaliações do Glassdoor de empresas Desenvolvimento de Software de Mato Grosso

Referências:
- [mato-grosso-tech-companies](https://github.com/leogregianin/mato-grosso-tech-companies?tab=readme-ov-file)
- Ecnodata
    - Filtro de empresas pelo CNAE `(J-6202-3/00) Desenvolvimento e licenciamento de programas de computador customizáveis` e pela Região `Cuiabá, Mato Grosso, Brasil`
    - https://www.econodata.com.br/empresas/mt-cuiaba/desenvolvimento-e-licenciamento-de-programas-de-computador-customizaveis-j-6202300
- Casa dos Dados
  - https://casadosdados.com.br/solucao/cnpj/pesquisa-avancada
- Glassdoor
  - https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/index.htm?overall_rating_low=1&page=1&locId=2467272&locType=C&locName=Cuiab%C3%A1,%20MT&occ=Information%20Technology&filterType=RATING_OVERALL

In [20]:
import logging
import time

from os import path
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [21]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("scraper.log"), logging.StreamHandler()],
)

In [22]:
# CHROME_PATH = "C:\chromedriver_win32_114_0_5735_90\chromedriver.exe"
HTML_PATH = "./html/"
PAGE_URLS = [
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Cigam-Software-Corporativo-Avalia%C3%A7%C3%B5es-E2485503.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/MTI-Empresa-Mato-grossense-de-Tecnologia-da-Informa%C3%A7%C3%A3o-Avalia%C3%A7%C3%B5es-E5151078.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Tecnomapas-Avalia%C3%A7%C3%B5es-E2492752.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Abaco-Tecnologia-de-Informa%C3%A7%C3%A3o-Ltda-Avalia%C3%A7%C3%B5es-E222140.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Liberali-Avalia%C3%A7%C3%B5es-E2651299.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Datweb-Avalia%C3%A7%C3%B5es-E4359774.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Funda%C3%A7%C3%A3o-Uniselva-Avalia%C3%A7%C3%B5es-E2494684.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Ganso-Sistemas-Avalia%C3%A7%C3%B5es-E2631184.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/MOVERE-Brazil-Avalia%C3%A7%C3%B5es-E2749942.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/DSS-Tecnologia-Avalia%C3%A7%C3%B5es-E2486780.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Eco-Centauro-Avalia%C3%A7%C3%B5es-E2021957.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Log-Lab-Intelig%C3%AAncia-Digital-Avalia%C3%A7%C3%B5es-E4509160.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Agenda-Assessoria-Avalia%C3%A7%C3%B5es-E2667223.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Grupo-Bom-Futuro-Avalia%C3%A7%C3%B5es-E2483312.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Datweb-Avalia%C3%A7%C3%B5es-E4359774.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/NBS-Inform%C3%A1tica-Avalia%C3%A7%C3%B5es-E2493877.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Amaggi-Avalia%C3%A7%C3%B5es-E42661.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Totem-Treinamento-e-Consultoria-em-TI-Avalia%C3%A7%C3%B5es-E2666668.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Redeflex-Avalia%C3%A7%C3%B5es-E2484166.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Original-Solu%C3%A7%C3%B5es-Tecnol%C3%B3gicas-Avalia%C3%A7%C3%B5es-E2769927.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Intelecto-Sistemas-Avalia%C3%A7%C3%B5es-E2675450.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Puma-Sistemas-Avalia%C3%A7%C3%B5es-E2639288.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/TGA-Sistemas-Avalia%C3%A7%C3%B5es-E2652853.htm", # Tangará da Serra
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Coplan-Consultoria-e-Planejamento-Avalia%C3%A7%C3%B5es-E2621617.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/TOTVS-Avalia%C3%A7%C3%B5es-E140298.htm", # nacional
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Plenusti-Avalia%C3%A7%C3%B5es-E8149411.htm",  # Primavera do Leste,
    "https://www.glassdoor.com/Reviews/Nuvem-Tecnologia-Reviews-E2799591.htm",
    # "https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Sankhya-Gest%C3%A3o-de-Neg%C3%B3cios-Avalia%C3%A7%C3%B5es-E1061278.htm",  # nacional
]

In [23]:
len(PAGE_URLS)

1

In [24]:
def write_html(html, file_name, replace=False):
    logging.info(f"Salvando arquivo html: {file_name}")

    file_exists = path.exists(file_name)
    if file_exists:
        logging.warning(f"Arquivo {file_name} já existe!")

    if not file_exists or replace:
        if file_exists and replace:
            logging.warning(f"Arquivo já {file_name} será sobrescrito!")

        with open(file_name, "w", encoding="utf-8") as f:
            f.write(html)

        logging.info(f"Arquivo {file_name} salvo com sucesso!")

In [25]:
def get_chrome_diver():
    service = Service()
    options = webdriver.ChromeOptions()
    return webdriver.Chrome(service=service, options=options)

In [26]:
def get_html_page(driver, page_url):
    try:
        driver.get(page_url)
        time.sleep(60)
        page_source = driver.page_source
    finally:
        driver.close()

    return page_source

In [27]:
START_FROM_PAGE_NUMBER = 1

In [28]:
for page_url in PAGE_URLS:
    page_number = START_FROM_PAGE_NUMBER
    next_page_url = page_url
    while True:
        driver = get_chrome_diver()

        logging.info(f"Baixando página: {next_page_url}")
        html_page = get_html_page(driver, next_page_url)

        soup = BeautifulSoup(html_page, "html.parser")

        logging.info("Buscando nome da empresa...")
        company_p = soup.find_all("p", {"class": "employerName"})
        if company_p:
            company = company_p[0].get_text()
            logging.info(f"Nome da empresa encontrado: {company}")
        else:
            logging.warning("Nome da empresa não encontrado!")

        company_clean = company.replace("(", "").replace(")", "")
        file_name = f"{company_clean}_page_{page_number}.html"

        write_html(html_page, HTML_PATH + file_name)

        logging.info("Buscando próxima página...")
        next_link = soup.find("link", rel="next")

        if next_link:
            next_page_url = next_link["href"]
            logging.info(f"Próxima página encontrada: {next_page_url}")
            page_number += 1
        else:
            logging.warning("Próxima página não encontrada!")
            break

2024-03-19 23:56:47,458 [INFO] Baixando página: https://www.glassdoor.com/Reviews/Nuvem-Tecnologia-Reviews-E2799591.htm
2024-03-19 23:57:52,816 [INFO] Buscando nome da empresa...
2024-03-19 23:57:52,818 [WARNING] Nome da empresa não encontrado!
2024-03-19 23:57:52,819 [INFO] Salvando arquivo html: ./html/Sankhya Gestão de Negócios_page_1.html
2024-03-19 23:57:52,820 [WARNING] Arquivo ./html/Sankhya Gestão de Negócios_page_1.html já existe!
2024-03-19 23:57:52,820 [INFO] Buscando próxima página...
2024-03-19 23:57:52,824 [WARNING] Próxima página não encontrada!
